In [1]:
import pandas as pd

In [2]:
from datetime import datetime, timedelta

In [3]:
auctions = pd.read_csv("auctions.csv")

In [4]:
events = pd.read_csv("events.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
installs = pd.read_csv("installs.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
auctions.head()

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [7]:
#ME QUEDO solo con los 3 primeros dias para uqe la distancia maxima sea de 3 dias
#arranca el 2019-04-18 00:00:00 asi que el date limite es 2019-04-20 23:59:59
limit_date = datetime(2019, 4, 20, 23, 59, 59, 999)

In [8]:
auctions["date"] = pd.to_datetime(auctions["date"])
auctions = auctions[auctions["date"] < limit_date]

In [9]:
#Agrupo por device_id, quedan los dates de apariciones en una lista
deviceIdDates = auctions.groupby(["device_id"])["date"].apply(list).to_frame().reset_index()

In [10]:
#Ordeno las fechas
deviceIdDates["date"] = deviceIdDates["date"].map(lambda x: sorted(x))

In [11]:
deviceIdDates.head()

,device_id,date
0,41863526108385,"[2019-04-19 19:40:28.465866, 2019-04-20 02:52:..."
1,135153013040192,"[2019-04-20 04:10:54.009137, 2019-04-20 04:18:..."
2,161514654074162,"[2019-04-18 02:52:46.357746, 2019-04-18 02:53:..."
3,181891380775191,[2019-04-20 23:19:25.420614]
4,186034136943920,"[2019-04-18 16:42:46.331894, 2019-04-18 19:23:..."


In [12]:
#Función que resta una fecha de aparición con la que le sigue inmediatamente
#limito a 500 el tamanio para que no itere tantas veces. sino itera muchas veces para los ids que aparecen
#muchas veces. hay muchos que aparecen miles de veces
def nextAppearanceTime( dateList ):
    distancias = []
    longitud_actual = len(dateList) if len(dateList)<500 else 500
    if(longitud_actual > 1):
        for x in range(longitud_actual):
            if(x + 1 < longitud_actual):
                distancias.append((pd.to_datetime(dateList[x + 1]) - pd.to_datetime(dateList[x])).total_seconds())
        
    return distancias

In [13]:
deviceIdDates["datesTimeDeltas"] = deviceIdDates["date"].map(lambda x: nextAppearanceTime(x))

In [14]:
#ahora hay una fila por cada device id.
#date es la lista de veces en las cuales aparecio
#y datestimedeltas es el tiempo promedio que tardo en volver a aparecer, y el valor a predecir en el futuro
deviceIdDates.head()

,device_id,date,datesTimeDeltas
0,41863526108385,"[2019-04-19 19:40:28.465866, 2019-04-20 02:52:...","[25918.427014, 395.61635, 419.166558, 175.7123..."
1,135153013040192,"[2019-04-20 04:10:54.009137, 2019-04-20 04:18:...","[480.352713, 27.220255, 1.312519, 80.049594, 6..."
2,161514654074162,"[2019-04-18 02:52:46.357746, 2019-04-18 02:53:...","[28.74402, 25.982516, 18.239779, 2.851261, 47...."
3,181891380775191,[2019-04-20 23:19:25.420614],[]
4,186034136943920,"[2019-04-18 16:42:46.331894, 2019-04-18 19:23:...","[9655.874138, 114849.7393, 22.040434, 17.51386..."


In [15]:
#hago que dates time deltas sea un numero en milisegundos
#si es una lista vacia(el id aparecio solamente en 1 auction) reemplazo por el tiempo en segundos equivalente 
# a 3 dias, ya que seria el maximo posible

In [16]:
three_days_seconds = timedelta(3).total_seconds()

In [17]:
deviceIdDates["datesTimeDeltas"] = deviceIdDates["datesTimeDeltas"].map(lambda x: three_days_seconds if len(x) == 0 else x[0])

In [18]:
#entreno asi nomas un modeo para despues poder usar como ejemplo

In [19]:
#creo un feature que diga si su primer aparecion fue dia de semana o no

In [20]:
deviceIdDates["first_appearence_is_weekend"] = deviceIdDates.apply(lambda row: row["date"][0].weekday()>4 , axis=1)

In [21]:
#x train va a tener los features ( en este caso solo uno)
X_train = deviceIdDates[["first_appearence_is_weekend"]]
#y train es el valor a predecir
y_train = deviceIdDates["datesTimeDeltas"]

In [22]:
#COMO LO QUE SE PREDICE ES TIEMPO USO UN REGRESOR
from sklearn.ensemble import RandomForestRegressor
#creo el regresor
clf = RandomForestRegressor()
#entreno el modeo
clf.fit(X_train, y_train)  


/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [23]:
#ahora invento un dataset, para el cual quiero predecir cuanto tiempo va a tardar en reaparecer
X_test = pd.DataFrame({"first_appearence_is_weekend": [True, False,True,True]})

In [24]:
#esta es la prediccion para cada registro  X test
clf.predict(X_test)

array([88002.76687758, 46269.49263662, 88002.76687758, 88002.76687758])

In [25]:
#buscando features

In [26]:
#en este df voy a ir agregando los nuevos features
df = deviceIdDates[["device_id", "first_appearence_is_weekend", "datesTimeDeltas"]]
df.head()

,device_id,first_appearence_is_weekend,datesTimeDeltas
0,41863526108385,False,25918.427014
1,135153013040192,True,480.352713
2,161514654074162,False,28.744020
3,181891380775191,True,259200.000000
4,186034136943920,False,9655.874138


In [27]:
#creo feature -> para cada id cual es la app mas usada por cada dispositivo a partir del dataframe EVENTS

In [ ]:
most_used_app = events.groupby("ref_hash")["application_id"].agg(lambda x: x.mode()).rename_axis("device_id").to_frame().reset_index()
most_used_app.head()

In [ ]:
import numpy as np

In [ ]:
#arreglo que algunos casos la app id mas usada es una lista. porque la funcion .mode() devolvio mas de un elemento
most_used_app["application_id"] = most_used_app["application_id"].apply(lambda x: x[0] if type(x) == np.ndarray else x)

In [ ]:
#lo agrego al df actual

In [ ]:
df = df.merge(most_used_app, on="device_id", how="left")

In [ ]:
#renombro para dejar mas claro que el nombre del nuevo feature
df = df.rename({"application_id": "most_used_app"}, axis="columns")

In [ ]:
#reemplazo los nan de most used app por un nuemero no existente
df["most_used_app"] = df["most_used_app"].fillna(-1)

In [ ]:
df.head()

In [ ]:
#creo feature para cada ID el sistema operativo a partir de dataframe Installs

In [ ]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [ ]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [ ]:
os = installs[["ref_hash", "os"]].dropna()

In [ ]:
#saco ids duplicados
os = os.drop_duplicates(subset="ref_hash", keep='first').rename({"ref_hash": "device_id"},axis="columns")

In [ ]:
#lo agrego a DF, siempre left join
df = df.merge(os,how="left", on="device_id")
df.head()

In [ ]:
#como hay muchos que no tienen info de sistema operativo, relleno los NA con "other"

In [ ]:
df["os"] = df["os"].fillna("other")

In [ ]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
#encodeo columna os
le.fit(df["os"])
df["os"] = le.transform(df["os"])

In [ ]:
#ahora en vez de string tengo sus codigos
df["os"].head()

In [ ]:

#CREO X_TRAIN Y Y_TRAIN
X_train = df[['first_appearence_is_weekend',
       'most_used_app', 'os']]
y_train = df['datesTimeDeltas']

In [ ]:
#entreno nuevo arbol

In [ ]:
clf = RandomForestRegressor()
#entreno el modeo
clf.fit(X_train, y_train)  


In [ ]:
#AHORA para predecir tengo que armar un X_test agarrando otros 3 dias del df original de auctions
#le hago las mismas transformaciones que fui haciendo arriba asi me queda un DF similar al de recien 
#y ahi puedo ver si estoy prediciendo bien